Generate data untuk mengecek waktu eksekusi berdasarkan jumlah node

In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import random
from itertools import permutations

# Definisi Kelas Komunitas

In [18]:
# number_of_node merupakan jumlah node pada komunitas

# number_of_relaton merupakan total koneksi pada komunitas tersebut
# nilai number_of_relation adalah 0 sampai 1
# artinya jika terdapat 3 node (A,B,C), maka total koneksi yang memungkinkan ada 6 (A-B, A-C, B-A, B-C, C-A, C-B)
# jika misalkan nilai number_of_relation adalah 0.5, maka akan diambil 3 koneksi dari total 6 koneksi

class Community:
  def __init__ (self, number_of_node, prob_connection=0):
    self.number_of_node = number_of_node
    self.prob_connection = prob_connection
    self.matrix_relation = 0
    self.table_of_node = Community.generate_table_node(self, self.number_of_node)
    self.table_of_relation = Community.generate_table_relation(self)

  # fungsi untuk generate tabel yang berisi daftar node pada suatu komunitas
  def generate_table_node(self, number_node):
    df = pd.DataFrame(columns=['Comm_ID','Node_ID','X','Y','Trust','RAM'])
    for i in range(number_node):
      df = df.append({'Comm_ID':'A', 'Node_ID':'A_' + str(i), 'X':random.randint(0,100),'Y':random.randint(0,100), 'Trust':round(random.uniform(0.5,1.0),3), 'RAM':random.uniform(0.1, 4.0)}, ignore_index = True)

    return df

  # fungsi untuk membuat matrik relasi node
  def generate_matrix_relation(self):
    number_node = self.number_of_node
    prob_connection = self.prob_connection
    mat_relation = np.identity(number_node, dtype=int) 
    all_con_posible = number_node -1
    if (prob_connection == 0):
      new_prob_con = []
      for i in range(number_node):
        new_prob_con.append(random.random())
      prob_connection = new_prob_con
    

    for i in range(number_node):
      list_row = list(mat_relation[i,:])

      list_pos_con = []
      for n,val in enumerate(list_row):
        if val == 0:
          list_pos_con.append(n)

      number_con = int(np.ceil(prob_connection[i] * all_con_posible))
      con_left=0
      
      print(len(list_pos_con))
      if (len(list_pos_con) == all_con_posible ):
        con_left = number_con
      else:
        con_left = number_con - (all_con_posible-len(list_pos_con))
      print("list_post_con ")
      print(list_pos_con)
      print("con_left ")
      print(con_left)
      #if (con_left > 0):
      rand_con = random.sample(list_pos_con, con_left)
      print('random connection')
      print(rand_con)
      print("\n")

      for j in rand_con:
        mat_relation[j,i] = 1
        mat_relation[i,j] = 1

    self.matrix_relation = mat_relation
    return mat_relation
  
  # menambahkan quality pada matrik connection
  def add_quality_relation(mat_relation):
    for i in range(len(mat_relation)):
      for n,val in enumerate(list(mat_relation[1,:])):
        if (n > i and val == 1):
          rand = round(random.uniform(0.5,1.0),3)
          mat_relation[n,i] = rand
          mat_relation[i,n] = rand    
    return mat_relation  

  # fungsi untuk generate tabel yang berisi daftar koneksi tiap node
  def generate_table_relation(self):
    number_node = self.number_of_node
    mat_relation = Community.generate_matrix_relation(self)

    mat_quality_relation = Community.add_quality_relation(mat_relation)
    print(mat_quality_relation)

    df = pd.DataFrame(columns=['node_1','node_2','quality','datetime'])

    for i in range(number_node):
      list_node_rel = list(mat_quality_relation[i,:])
      for n,val in enumerate(list_node_rel):
        if (n != i and val != 0):
          df = df.append({'node_1':'A_'+ str(i), 'node_2':'A_' + str(n), 'quality':val,'datetime':dt.datetime.now()}, ignore_index = True)

    return df

  #fungsi untuk kalkulasi
  def calculate_table(self):
    df_calculate = self.table_of_relation.groupby(['node_1']).sum()
    df_calculate['number_of_relation'] = list(self.table_of_relation.groupby(['node_1']).size())
    df_calculate['rate_quality'] = df_calculate['quality']/df_calculate['number_of_relation']
    
    df_node = self.table_of_node.copy()
    df_node['comp_rate'] = df_node['RAM'].apply(Community.parse_ram)

    df_calculate['comp_rate'] = df_node['comp_rate'].tolist()
    return df_calculate.sort_values(by='rate_quality', ascending=False)

  def parse_ram(ram):
    pwr = 0
    if ram < 1 :
        pwr = 0.3
    if ram >= 1 :
        pwr = 0.6
    if ram >=4 :
        pwr = 1
    return pwr

## Buat objek komunitas

In [21]:
coba = Community(number_of_node=5, prob_connection=[0.1, 0.3, 0.5, 0.4, 0.5])

4
list_post_con 
[1, 2, 3, 4]
con_left 
4
random connection
[1, 4, 3, 2]


3
list_post_con 
[2, 3, 4]
con_left 
3
random connection
[2, 4, 3]


2
list_post_con 
[3, 4]
con_left 
2
random connection
[4, 3]


1
list_post_con 
[4]
con_left 
1
random connection
[4]


0
list_post_con 
[]
con_left 
0
random connection
[]


[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 1 1]
 [0 0 1 1 1]
 [0 0 1 1 1]]


In [7]:
coba.matrix_relation

array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 1]])

In [101]:
coba.table_of_relation

,node_1,node_2,quality,datetime
0,A_0,A_3,1,2021-10-04 11:47:15.315845
1,A_2,A_3,1,2021-10-04 11:47:15.318699
2,A_2,A_4,1,2021-10-04 11:47:15.321708
3,A_3,A_0,1,2021-10-04 11:47:15.324800
4,A_3,A_2,1,2021-10-04 11:47:15.328165
5,A_4,A_2,1,2021-10-04 11:47:15.331135


In [ ]:
coba.calculate_table()

,quality,number_of_relation,rate_quality,comp_rate
node_1,,,,
A4,3.092857,4,0.773214,0.6
A0,2.188668,4,0.547167,0.3
A1,1.873204,4,0.468301,0.6
A2,1.752125,4,0.438031,0.6
A3,1.581499,4,0.395375,0.6


In [62]:
import numpy as np

coba_np = np.identity(5)
a = [
     [1,2,3,4,5],
     [6,7,8,9,10],
     [11,12,13,14,15],
     [16,17,18,19,20],
     [21,22,23,24,25]
     ]
np_a = np.array(a)
coba_np

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [4]:
prob_coba = [0.1, 0.3, 0.5, 0.4, 0.5]

for i in range(5):
  print(np_a[i][0])
  np_a[i][0] *= 5 

np_a

1
6
11
16
21


array([[  5,   2,   3,   4,   5],
       [ 30,   7,   8,   9,  10],
       [ 55,  12,  13,  14,  15],
       [ 80,  17,  18,  19,  20],
       [105,  22,  23,  24,  25]])

In [23]:
for i in range(5):
  list_con = []
  number_con = int(np.ceil(prob_coba[i] * 5))
  for j in range(number_con):
    numb_ran = random.randint(0,4)
    if (numb_ran not in list_con and numb_ran != i):
      list_con.append(numb_ran)
    elif (numb_ran in list_con or numb_ran == i):
      while (numb_ran in list_con or numb_ran == i):
        numb_ran = random.randint(0,4)
      list_con.append(numb_ran)
  print(number_con)
  print(list_con)
  print('=============')
  for j in list_con:
    coba_np[i][j] = j+1

1
[2]
2
[0, 2]
3
[0, 4, 1]
2
[4, 1]
3
[3, 1, 0]


In [6]:
coba_np

array([[1, 2, 0, 0, 0],
       [1, 1, 0, 0, 5],
       [1, 2, 1, 4, 0],
       [0, 2, 3, 1, 0],
       [1, 2, 0, 4, 1]])

In [63]:
for i in range(5):
  list_row = list(coba_np[i,:])

  list_pos_con = []
  for n,val in enumerate(list_row):
    if val == 0:
      list_pos_con.append(n)

  print(list_pos_con)
  number_con = int(np.ceil(prob_coba[i] * 4))
  con_left=0
  if (len(list_pos_con) == 4 ):
    con_left = number_con
  else:
    con_left = number_con - (4-len(list_pos_con))
  print("list_post_con ")
  print(list_pos_con)
  print("con_left ")
  print(con_left)
  print("\n")
  rand_con = random.sample(list_pos_con, con_left)

  for j in rand_con:
    coba_np[j,i] = 1
    coba_np[i,j] = 1

coba_np

[1, 2, 3, 4]
1
[0, 2, 3, 4]
2
[3, 4]
0
[0, 1, 2, 4]
2
[0, 2, 3]
1


array([[1., 0., 1., 1., 1.],
       [0., 1., 1., 1., 1.],
       [1., 1., 1., 0., 0.],
       [1., 1., 0., 1., 0.],
       [1., 1., 0., 0., 1.]])

In [64]:
for i in range(len(coba_np)):
  for n,val in enumerate(list(coba_np[1,:])):
    if (n > i and val == 1):
      rand = round(random.random(),3)
      coba_np[n,i] = rand
      coba_np[i,n] = rand

coba_np

array([[1.   , 0.875, 0.734, 0.874, 0.662],
       [0.875, 1.   , 0.791, 0.151, 0.167],
       [0.734, 0.791, 1.   , 0.   , 0.   ],
       [0.874, 0.151, 0.   , 1.   , 0.   ],
       [0.662, 0.167, 0.   , 0.   , 1.   ]])

In [56]:
coba_np

array([[1.   , 0.024, 0.   , 0.354, 0.   ],
       [0.024, 1.   , 0.   , 0.772, 0.   ],
       [0.   , 0.   , 1.   , 1.   , 1.   ],
       [0.354, 0.772, 1.   , 1.   , 1.   ],
       [0.   , 0.   , 1.   , 1.   , 1.   ]])

In [65]:
df = pd.DataFrame(columns=['node_1','node_2','quality','datetime'])
for i in range(len(coba_np)):
  list_node_rel = list(coba_np[i,:])
  for n,val in enumerate(list_node_rel):
    if (n != i and val != 0):
      df = df.append({'node_1':'A_'+ str(i), 'node_2':'A_' + str(n), 'quality':val,'datetime':dt.datetime.now()}, ignore_index = True)

In [66]:
df

,node_1,node_2,quality,datetime
0,A_0,A_1,0.875,2021-10-04 10:03:32.598448
1,A_0,A_2,0.734,2021-10-04 10:03:32.602340
2,A_0,A_3,0.874,2021-10-04 10:03:32.604916
3,A_0,A_4,0.662,2021-10-04 10:03:32.607580
4,A_1,A_0,0.875,2021-10-04 10:03:32.610289
5,A_1,A_2,0.791,2021-10-04 10:03:32.612808
6,A_1,A_3,0.151,2021-10-04 10:03:32.615407
7,A_1,A_4,0.167,2021-10-04 10:03:32.617877
8,A_2,A_0,0.734,2021-10-04 10:03:32.620400
9,A_2,A_1,0.791,2021-10-04 10:03:32.622981


In [68]:
a = 0
a = [1,2,3,4,5]
a

[1, 2, 3, 4, 5]

In [90]:
round(random.uniform(0.5,1.0),3)

0.629